# day 20

https://adventofcode.com/2020/day/20

In [ ]:
import logging
import logging.config
import os

import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day20.txt')

LOGGER = logging.getLogger('day20')

## part 1

### problem statement:

#### loading data

In [ ]:
test_data = """Tile 2311:
..##.#..#.
##..#.....
#...##..#.
####.#...#
##.##.###.
##...#.###
.#.#.#..##
..#....#..
###...#.#.
..###..###

Tile 1951:
#.##...##.
#.####...#
.....#..##
#...######
.##.#....#
.###.#####
###.##.##.
.###....#.
..#.#..#.#
#...##.#..

Tile 1171:
####...##.
#..##.#..#
##.#..#.#.
.###.####.
..###.####
.##....##.
.#...####.
#.##.####.
####..#...
.....##...

Tile 1427:
###.##.#..
.#..#.##..
.#.##.#..#
#.#.#.##.#
....#...##
...##..##.
...#.#####
.#.####.#.
..#..###.#
..##.#..#.

Tile 1489:
##.#.#....
..##...#..
.##..##...
..#...#...
#####...#.
#..#.#.#.#
...#.#.#..
##.#...##.
..##.##.##
###.##.#..

Tile 2473:
#....####.
#..#.##...
#.##..#...
######.#.#
.#...#.#.#
.#########
.###.#..#.
########.#
##...##.#.
..###.#.#.

Tile 2971:
..#.#....#
#...###...
#.#.###...
##.##..#..
.#####..##
.#..####.#
#..#.#..#.
..####.###
..#.#.###.
...#.#.#.#

Tile 2729:
...#.#.#.#
####.#....
..#.#.....
....#..#.#
.##..##.#.
.#.####...
####.#.#..
##.####...
##..#.##..
#.##...##.

Tile 3079:
#.#.#####.
.#..######
..#.......
######....
####.#..#.
.#...#.##.
#.#####.##
..#.###...
..#.......
..#.###..."""

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        #return [line.strip() for line in fp]
        return fp.read().strip()

In [ ]:
import numpy as np
def parse_data(data):
    d = {}
    for tile in data.strip().split('\n\n'):
        id_line, *tile_vals = tile.split('\n')
        tile_id = int(id_line.split(' ')[1][:-1])
        a = np.array([[c == '#' for c in row] for row in tile_vals])
        d[tile_id] = a
    return d

In [ ]:
from collections import defaultdict

def get_possible_edges(d):
    edge_dict = defaultdict(set)
    for (tile_id, a) in d.items():
        edges = [a[0, :],
                 a[:, 0],
                 a[-1, :],
                 a[:, -1]]
        for edge in edges:
            edge_rep_a = tuple(edge)
            edge_rep_b = tuple(edge[::-1])
            edge_dict[min(edge_rep_a, edge_rep_b)].add(tile_id)

    tile_map = defaultdict(dict)
    for edge, tile_set in edge_dict.items():
        for tile_0 in tile_set:
            for tile_1 in tile_set:
                if tile_0 != tile_1:
                    tile_map[tile_0][tile_1] = edge
                    
    return edge_dict, tile_map

In [ ]:
d = parse_data(test_data)
get_possible_edges(d)

#### function def

In [ ]:
def q_1(data):
    d = parse_data(data)
    possible_edges, tile_map = get_possible_edges(d)
    # shortcut:
    only_corners = {k for (k, v) in tile_map.items() if len(v) == 2}
    if len(only_corners) == 4:
        x = 1
        for tile_id in only_corners:
            x *= tile_id
        return x
    raise ValueError()
    return d, possible_edges

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) == 20899048083289
    LOGGER.setLevel(logging.INFO)

In [ ]:
q_1(test_data)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

#### function def

In [ ]:
def q_2(data):
    d = parse_data(data)
    possible_edges, tile_map = get_possible_edges(d)
    interiors = {tile_id: d[tile_id]
                 for (tile_id, neighbor_set) in tile_map.items()
                 if len(neighbor_set) == 4}
    possible_occupied = sum(a.sum() for a in interiors.values())
    snake_cts = [possible_occupied - 15 * i for i in range(2, 10)]
    return d, possible_edges, tile_map, interiors, possible_occupied, snake_cts

In [ ]:
d, possible_edges, tile_map, interiors, possible_occupied, snake_cts = q_2(load_data())
size = len(tile_map) ** .5
size

In [ ]:
interiors

In [ ]:
import networkx as nx

In [ ]:
tile_map

In [ ]:
g = nx.Graph()
for src, dst_dict in tile_map.items():
    for dst in dst_dict:
        g.add_edge(src, dst)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
logging.getLogger('matplotlib').setLevel(logging.ERROR)
fig, ax = plt.subplots(1, 1, figsize=(15, 15))
nx.draw_kamada_kawai(g, with_labels=True, font_color='red', font_size=16, node_color='white')

In [ ]:
box = [
    [1759, 2287, 2663, 2609, 3769, 3637, 2011, 2273, 1699, 3593, 3529, 2719],
    [1069, 2003, 2671, 2879, 3467, 3449, 3251, 1607, 1459, 1811, 1409, 3271],
    [2917, 1481, 3833, 1061, 1549, 1423, 3049, 3557, 1913, 2729, 1787, 1609],
    [1721, 3347, 3371, 1973, 2129, 2803, 2459, 1523, 1783, 1801, 1553, 3881],
    [1063, 2269, 2711, 3797, 1559, 1861, 1109, 1901, 3917, 2437, 2791, 2381],
    [3259, 2063, 3533, 3079, 1531, 3307, 2819, 3191, 2789, 1153, 3919, 1087],
    [1307, 1667, 1187, 1831, 1583, 1301, 2237, 2699, 3041, 2549, 2887, 1949],
    [3037, 1747, 3203, 1051, 1907, 2473, 2399, 3659, 1283, 2591, 1823, 1889],
    [2713, 1847, 1933, 2777, 1427, 3119, 2749, 2347, 3821, 1279, 1697, 2939],
    [1471, 1367, 2909, 2081, 3461, 3889, 3499, 2539, 3709, 3373, 3947, 3169],
    [1319, 1093, 2281, 3793, 2797, 2089, 1163, 1019, 1979, 1447, 1657, 3607],
    [2801, 2203, 3697, 1249, 1031, 1867, 1303, 1723, 2411, 1871, 3727, 3823],
]

In [ ]:
import itertools

def transform_a(a, reflect_x, reflect_y, transpose):
    if reflect_x:
        a = np.flip(a, 0)
    if reflect_y:
        a = np.flip(a, 1)
    if transpose:
        a = a.T
    return a

def get_orientations(a):
    a = a.copy()
    for xyt in itertools.product([True, False], [True, False], [True, False]):
        yield transform_a(a, *xyt), xyt

In [ ]:
for i, ((a_lft, xyt_lft), (a_rgt, xyt_rgt)) in enumerate(itertools.combinations(list(get_orientations(d[1759])), 2)):
    if (a_lft == a_rgt).all():
        print(xyt_lft, xyt_rgt)

In [ ]:
# 8: transpose, reflect x, refect y

In [ ]:
# fix the orientation of 2003 by hand first

In [ ]:
print(f'1759 right to 2287: {tile_map[1759][2287]}')
print(f'1759 down to 1069: {tile_map[1759][1069]}')

In [ ]:
for (i, (a, xyt)) in enumerate(get_orientations(d[1759])):
    if a[:, -1].sum() == 5 and a[-1, :].sum() == 4:
        print(i)
        print(a)
        known_tiles = {}
        known_tiles[0, 0] = a

In [ ]:
for (i, row) in enumerate(box):
    for (j, tile_id) in enumerate(row[:-1]):
        # there can be only one way to connect this to the one to its right; choose it
        tile_id_right = row[j + 1]
        a = known_tiles[i, j]
        a_right_orig = d[tile_id_right]
        
        for (a_right, xyt_right) in get_orientations(a_right_orig):
            if (a[:, -1] == a_right[: ,0]).all():
                known_tiles[i, j + 1] = a_right
                break
        
        if (j == 0) and (i != (len(box) - 1)):
            tile_id_down = box[i + 1][j]
            a_down_orig = d[tile_id_down]
            for (a_down, xyt_down) in get_orientations(a_down_orig):
                if (a[-1, :] == a_down[0, :]).all():
                    known_tiles[i + 1, j] = a_down
                    break

In [ ]:
# assemble
z = np.zeros((8 * 12, 8 * 12), bool)

for ((i, j), tile) in known_tiles.items():
    i0 = 8 * i
    j0 = 8 * j
    z[i0: i0 + 8, j0: j0 + 8] = tile[1: 9, 1: 9]


ax, fig = plt.subplots(1, 1, figsize=(15, 15))
fig.imshow(z)

In [ ]:
MONSTER = """                  # 
#    ##    ##    ###
 #  #  #  #  #  #   """.replace(' ', '.')
MONSTER = np.array([[c == '#' for c in row] for row in MONSTER.split('\n')])

ax, fig
plt.imshow(MONSTER)

In [ ]:
# find sea creatures
def find_monsters(img, monster):
    for i0 in range(img.shape[0] - monster.shape[0]):
        for j0 in range(img.shape[1] - monster.shape[1]):
            sub_img = img[i0: i0 + monster.shape[0],
                          j0: j0 + monster.shape[1]]
            if (monster == (sub_img & monster)).all():
                yield i0, j0

In [ ]:
# test case
img = """.####...#####..#...###..
#####..#..#.#.####..#.#.
.#.#...#.###...#.##.O#..
#.O.##.OO#.#.OO.##.OOO##
..#O.#O#.O##O..O.#O##.##
...#.#..##.##...#..#..##
#.##.#..#.#..#..##.#.#..
.###.##.....#...###.#...
#.####.#.#....##.#..#.#.
##...#..#....#..#...####
..#.##...###..#.#####..#
....#.##.#.#####....#...
..##.##.###.....#.##..#.
#...#...###..####....##.
.#.##...#.##.#.#.###...#
#.###.#..####...##..#...
#.###...#.##...#.##O###.
.O##.#OO.###OO##..OOO##.
..O#.O..O..O.#O##O##.###
#.#..##.########..#..##.
#.#####..#.#...##..#....
#....##..#.#########..##
#...#.....#..##...###.##
#..###....##.#...##.##.#""".replace('O', '#')

img = np.array([[c == '#' for c in row] for row in img.split('\n')])

N = len(list(find_monsters(img, MONSTER)))
assert N == 2

print(img.sum() - N * MONSTER.sum())

In [ ]:
for z_prime, xyt_prime in get_orientations(z):
    monsters = list(find_monsters(z_prime, MONSTER))
    N = len(monsters)
    print(N)
    if N != 0:
        print(f"answer: {z_prime.sum() - N * MONSTER.sum()}")
        break

let's see them monsters!

In [ ]:
plottable = z_prime.astype(int)
for (i0, j0) in monsters:
    plottable[i0: i0 + MONSTER.shape[0], j0: j0 + MONSTER.shape[1]] += MONSTER

ax, fig = plt.subplots(1, 1, figsize=(15, 15))
fig.imshow(plottable, cmap='cividis')

#### tests

In [ ]:
# def test_q_2():
#     LOGGER.setLevel(logging.DEBUG)
#     assert q_2(test_data) == True
#     LOGGER.setLevel(logging.INFO)

In [ ]:
# test_q_2()

#### answer

In [ ]:
# q_2(load_data())

fin